In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hiplot as hip
import pandas as pd
import time
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, Matern, ExpSineSquared,DotProduct
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from yellowbrick.features import ParallelCoordinates

In [ ]:
#Reading Input

df1 = pd.read_csv('Input.csv')

df1.rename(columns = {'TLI':'InletLiquidTemperature', 'TAI':'InletAirTemperature'
                   , 'H':'PackedBedHeight' , 'D':'PackedBedDiameter', 'ML':'LiquidMassFlow'
                   , 'MA':'AirMassFlow', 'EPS':'PackedBedPorosity'}, inplace = True)

X = df1.to_numpy() #Input

#Reading Output

df2 = pd.read_csv('Output.csv')

df2.rename(columns = {'TLO':'OutletLiquidTemperature', 'TAO':'OutletAirTemperature'}, inplace = True)

Y = df2.to_numpy() 

df3 = pd.concat([df1, df2], axis=1, join="inner")

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
# Best linear estimation

reg = MultiOutputRegressor(LinearRegression()).fit(X, Y)
linear_score=reg.score(X, Y)
print(linear_score)

In [ ]:
trainsize=0.8
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=trainsize)

In [ ]:
# transform data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)
Y0_train = Y_train[:,0]
Y1_train = Y_train[:,1]
Y0_test = Y_test[:,0]
Y1_test = Y_test[:,1]

In [ ]:
Arr = []
Arr_df = pd.DataFrame(Arr,columns = ['Train Size','First layer','Second layer','Learning Rate',
                                     'Solver','Activation','MLP Training','MLP Test'])
for i in [0.001,0.01,0.1]:
    for j in ['sgd','lbfgs','adam']:
        for k in ['relu','tanh','logistic','identity']:
            for l in [0.0001,0.001,0.01,0.1]:
                for m in [10000,20000,40000,80000]:
                    mlp = MultiOutputRegressor(MLPRegressor(hidden_layer_sizes=(6,3), max_iter=m,
                                solver=j, verbose=0, tol=1e-5,activation = k,learning_rate_init=i,alpha=l))
                    mlp.fit(X_train_scaled, Y_train)
                    Arr.append([trainsize, i,j,k,l,m,mlp.score(X_train_scaled, Y_train),mlp.score(X_test_scaled, Y_test)])
                    print(i,j,k,l,m)

In [ ]:
NN_Result = pd.DataFrame(Arr, columns = ['Train Size','Learning Rate','Solver ','Activation',
                                     'Alpha','Max Iteration','MLP Training','MLP Test'])

In [ ]:
print(NN_Result)

In [ ]:
from pathlib import Path  
filepath = Path('MLP_Hyper.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
NN_Result.to_csv(filepath)

In [ ]:
NN_Result.loc[NN_Result['MLP Test'] >= 0.99]

In [ ]:
Arr1 = []
Arr_df = pd.DataFrame(Arr1,columns = ['Train Size','First layer','Second layer','Third layer'
                                      ,'MLP0 Training','MLP0 Test','MLP1 Training','MLP1 Test'])
for i in range(1,10):
    for j in range(1,10):
        for k in range(1,10):
            
            mlp0 = MLPRegressor(hidden_layer_sizes=(i,j,k), max_iter=40000, alpha=1,
                    solver='sgd', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)
            mlp1 = MLPRegressor(hidden_layer_sizes=(i,j,k), max_iter=40000, alpha=1,
                    solver='sgd', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)
            mlp0.fit(X_train_scaled, Y0_train)
            mlp1.fit(X_train_scaled, Y1_train)
            Arr1.append([trainsize, i,j,k,mlp0.score(X_train_scaled, Y0_train),mlp0.score(X_test_scaled, Y0_test)
                         ,mlp1.score(X_train_scaled, Y1_train),mlp0.score(X_test_scaled, Y1_test)])
            print(i,j,k)

In [ ]:
Arr1

In [ ]:
Arr = []
Arr_df = pd.DataFrame(Arr,columns = ['Train Size','First layer','Second layer','Third layer','MLP Training','MLP Test'])
time_start = time.time()
for i in range(1,10):
    for j in range(1,10):
        for k in range(1,10):
            mlp = MultiOutputRegressor(MLPRegressor(hidden_layer_sizes=(i,j,k), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01),n_jobs=40)
            mlp.fit(X_train_scaled, Y_train)
            Arr.append([trainsize, i,j,k,mlp.score(X_train_scaled, Y_train),mlp.score(X_test_scaled, Y_test)])
            print(i,j,k,mlp.score(X_train_scaled, Y_train),mlp.score(X_test_scaled, Y_test))
    
time_elapsed = (time.time() - time_start)
print('time_elapsed=', time_elapsed)

In [ ]:
NN_Result_3Layers = pd.DataFrame(Arr, columns = ['Train Size','First layer','Second layer',
                                                 'Third layer','MLP Training','MLP Test'])
from pathlib import Path  
filepath = Path('NN_Result_3Layers.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
NN_Result_3Layers.to_csv(filepath)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [ ]:
my_stand_scaler_obj = StandardScaler()
my_stand_scaler_obj.fit(X_train)

# rescale the training data
X_stand_scaler = my_stand_scaler_obj.transform(X)

# rescale the training data
X_train_stand_scaler = my_stand_scaler_obj.transform(X_train)

# also, rescale the training data
X_test_stand_scaler = my_stand_scaler_obj.transform(X_test)


print(my_stand_scaler_obj.mean_)

In [ ]:
#KRR  = KernelRidge(alpha=.01, kernel='rbf')
MLP1 = MLPRegressor(hidden_layer_sizes=(3,3), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)
MLP2 = MLPRegressor(hidden_layer_sizes=(6,6), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)
MLP3 = MLPRegressor(hidden_layer_sizes=(9,9), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)
MLP4 = MLPRegressor(hidden_layer_sizes=(3,3,3), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)
MLP5 = MLPRegressor(hidden_layer_sizes=(6,6,6), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)
MLP6 = MLPRegressor(hidden_layer_sizes=(9,9,9), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(MLP1,
                                                        X_stand_scaler, Y,
                                                        cv = 10,
                                                        scoring="r2", 
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

# Means and standard deviations of training set scores
train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)
# Means and standard deviations of test set scores
test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

In [ ]:
Arr=np.linspace(0.1, 0.9, 51).reshape(-1,1)

In [ ]:
Arr

In [ ]:
Arr=np.append(Arr,train_mean1,axis=1)

In [ ]:
Arr=np.append(Arr,test_mean1,axis=1)

In [ ]:
Arr

In [ ]:
#Reading Input
df1 = pd.read_csv('Input4.csv')
df1.rename(columns = {'TLI':'InletLiquidTemperature', 'TAI':'InletAirTemperature'
                   , 'H':'PackedBedHeight' , 'D':'PackedBedDiameter', 'ML':'LiquidMassFlow'
                   , 'MA':'AirMassFlow', 'EPS':'PackedBedPorosity'}, inplace = True)

X = df1.to_numpy() #Input
#Reading Output
df2 = pd.read_csv('Output4.csv')
df2.rename(columns = {'TLO':'OutletLiquidTemperature', 'TAO':'OutletAirTemperature'}, inplace = True)
Y = df2.to_numpy() 

from sklearn.multioutput import MultiOutputRegressor
# Best linear estimation

reg = MultiOutputRegressor(LinearRegression()).fit(X, Y)
linear_score=reg.score(X, Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
# transform data
my_stand_scaler_obj = StandardScaler()
my_stand_scaler_obj.fit(X_train)

# rescale the training data
X_stand_scaler = my_stand_scaler_obj.transform(X)

# rescale the training data
X_train_stand_scaler = my_stand_scaler_obj.transform(X_train)

# also, rescale the training data
X_test_stand_scaler = my_stand_scaler_obj.transform(X_test)

Arr=[]

########################################################################################################################################################

MLP1 = MLPRegressor(hidden_layer_sizes=(3,3), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP1,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_sizes1 = train_sizes.reshape(-1,1)

Arr= train_sizes1

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP1,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################

MLP2 = MLPRegressor(hidden_layer_sizes=(3,6), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP2,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP2,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################

MLP3 = MLPRegressor(hidden_layer_sizes=(6,3), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP3,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP3,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################

MLP4 = MLPRegressor(hidden_layer_sizes=(3,3,3), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP4,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP4,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################

MLP5 = MLPRegressor(hidden_layer_sizes=(3,3,6), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP5,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP5,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################

MLP6 = MLPRegressor(hidden_layer_sizes=(6,3,3), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP6,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP6,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################
                    
MLP7 = MLPRegressor(hidden_layer_sizes=(6,3,6), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP7,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP7,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################

MLP8 = MLPRegressor(hidden_layer_sizes=(6,6,6), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP8,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(1111111111111111111111111111111111111111111111111111111111)

train_sizes, train_scores, test_scores = learning_curve(MLP8,X_stand_scaler,
                                                        Y,cv = 10,scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

print(2222222222222222222222222222222222222222222222222222222222)

########################################################################################################################################################

MLP_8Config = pd.DataFrame(Arr)
from pathlib import Path  

filepath = Path('MLP_8Config.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
MLP_8Config.to_csv(filepath)

In [ ]:
Arr=[]
MLP1 = MLPRegressor(hidden_layer_sizes=(3,3), max_iter=40000, alpha=1,solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01)

train_sizes, train_scores, test_scores = learning_curve(MLP1,X_stand_scaler,
                                                        Y,cv = 10,scoring="neg_mean_squared_error",
                                                        train_sizes = np.linspace(0.1, 0.9, 51))

train_sizes1 = train_sizes.reshape(-1,1)

Arr= train_sizes1

train_mean1 = np.mean(train_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

test_mean1 = np.mean(test_scores, axis=1).reshape(-1,1)

Arr=np.append(Arr,train_mean1,axis=1)

In [ ]:
Arr

In [ ]:
Arr1 = pd.DataFrame(Arr)

Arr1[0]

In [ ]:
Arr1.rename(columns = {'0':'Train Size','2':'Test_MSE (3,3)'})

Arr1

In [ ]:
fig = plt.figure(figsize=(9, 6))
plt.plot(Arr1[0],Arr1[1])

In [ ]:
mlp6 = MultiOutputRegressor(MLPRegressor(hidden_layer_sizes=(6,6,6), max_iter=40000, alpha=1,
                    solver='lbfgs', verbose=0, tol=1e-5,activation = 'relu',learning_rate_init=.01))

In [ ]:
time_start = time.time()
mlp6.fit(X_train_scaled, Y_train)
time_elapsed = (time.time()-time_start)